In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense
from qkeras import QDense, quantized_bits, QActivation


In [ ]:
# Definicja modelu, wzór
model = Sequential([
    Flatten(input_shape=(28, 28)),
    Dense(200, activation='sigmoid'),
    Dense(80, activation='sigmoid'),
    Dense(10, activation='sigmoid')
])

In [ ]:
from tensorflow.keras.datasets import mnist

# Pobieranie danych MNIST
(X_train, y_train), (X_test, y_test) = mnist.load_data()



# Sprawdzenie kształtów danych
print("Kształt X_train:", X_train.shape)
print("Kształt y_train:", y_train.shape)
print("Kształt X_test:", X_test.shape)
print("Kształt y_test:", y_test.shape)


In [ ]:
model = Sequential([
    Flatten(input_shape=(28, 28)),
    Dense(200, activation='sigmoid'),
    Dense(80, activation='sigmoid'),
    Dense(10, activation='sigmoid')
])


In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=3, validation_data=(X_test, y_test))

Analizujemy kolejne stopnie kwantyzacji, każdy po 5 razy żeby można było policzyć statystykę. Za każdym razem na nowo losujemy podział train_test_split

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
acc = np.zeros((8, 5))
for j in range(0, 5):
    #robimy tak bo zbiór z tensorflow jest domyślnie podzielony na test i train, a my chcemy to podzielić losowo
    X_full = np.concatenate((X_train, X_test))
    y_full = np.concatenate((y_train, y_test))

    X_train, X_test, y_train, y_test = train_test_split(X_full, y_full, test_size=0.2)
    for i in range (1, 9):
        q_bits = i
        q_model = Sequential([
            Flatten(input_shape=(28, 28)),
            QDense(200, input_shape=(28*28,), activation='sigmoid', 
                kernel_quantizer=quantized_bits(q_bits, 0, 1), 
                bias_quantizer=quantized_bits(q_bits, 0, 1)),
            QDense(80, activation='sigmoid', 
                kernel_quantizer=quantized_bits(q_bits, 0, 1), 
                bias_quantizer=quantized_bits(q_bits, 0, 1)),
            QDense(10, activation='sigmoid', 
                kernel_quantizer=quantized_bits(q_bits, 0, 1), 
                bias_quantizer=quantized_bits(q_bits, 0, 1))
        ])

        q_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
        q_model.fit(X_train, y_train, epochs=3, validation_data=(X_test, y_test))
        acc[i-1, j] = q_model.evaluate(X_test, y_test, verbose=0)[1]
        print(acc[i-1, j])